In [6]:
import oss2
import json
import cv2
import matplotlib.pyplot as plt
import requests
%matplotlib inline
import numpy as np
with open("../aliyun_key.json" ,'r') as key:
    AccessKeyDict=json.loads(key.read())
AccessKeyId=AccessKeyDict["AccessKeyId"]
AccessKeySecret=AccessKeyDict["AccessKeySecret"]
auth = oss2.Auth(AccessKeyId, AccessKeySecret)
bucket = oss2.Bucket(auth, 'https://oss-cn-shanghai.aliyuncs.com', 'ocr-yjs')
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkcore.auth.credentials import AccessKeyCredential
from aliyunsdkcore.auth.credentials import StsTokenCredential
credentials = AccessKeyCredential(AccessKeyId, AccessKeySecret)
client = AcsClient(region_id='cn-shanghai', credential=credentials)
import seaborn as sns

In [3]:
"""

https://help.aliyun.com/document_detail/188844.html
人流热度图估计

人流密集场所人数统计：在举办某些人流密度较高的活动时，例如庙会、烟火表演、
演唱会等，通过实时图像可以估算出现场人数，为现场安保提供数据支撑。
商场货品陈列：通过统计叠加一段时间内的人流热度图，可以获悉哪些区域有更多人流经过，
推断客户对哪些商品更感兴趣，为商品陈列做参考，以获得更高客单价。
"""

from aliyunsdkfacebody.request.v20191230.CountCrowdRequest import CountCrowdRequest

request = CountCrowdRequest()
request.set_accept_format('json')

filepath="images/manyperson.jpg"
with open(filepath, 'rb') as fileobj:
    bucket.put_object(filepath, fileobj)
url="https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/"+filepath
request.set_ImageURL(url)
request.set_IsShow(True)

response = client.do_action_with_exception(request)
response=json.loads(str(response, encoding='utf-8'))
print(response)

{'RequestId': 'BE7D8EE0-0448-591E-B7E9-310AEBE32B87', 'Data': {'PeopleNumber': 7, 'HotMap': 'http://vibktprfx-prod-prod-aic-iv-cn-shanghai.oss-cn-shanghai.aliyuncs.com/crowd-counting%2F084e29c19a44451eafb7d0a0dd648958.jpg?OSSAccessKeyId=LTAI4G4YbtUh1c7BhUoTTAzS&Expires=1629530913&Signature=Fz3VNjy2Wnbt4B72XVP302DZINI%3D'}}


In [4]:
PeopleNumber=response["Data"]["PeopleNumber"]
PeopleNumber

7

In [5]:
HotMap=response["Data"]["HotMap"]
HotMap

'http://vibktprfx-prod-prod-aic-iv-cn-shanghai.oss-cn-shanghai.aliyuncs.com/crowd-counting%2F084e29c19a44451eafb7d0a0dd648958.jpg?OSSAccessKeyId=LTAI4G4YbtUh1c7BhUoTTAzS&Expires=1629530913&Signature=Fz3VNjy2Wnbt4B72XVP302DZINI%3D'

In [7]:
with open("./images/HotMap.jpg",'wb') as file:
    file.write(requests.get(response["Data"]["HotMap"]).content)